In [44]:
import requests
import urllib.request
import time
import dbInterface
from bs4 import BeautifulSoup
import datetime
from datetime import datetime
from bs4.dammit import EncodingDetector
import locale

In [45]:
monthDict={
    "Januar":1,
    "Februar":2,
    "März":3,
    "April":4,
    "Mai":5,
    "Juni":6,
    "Juli":7,
    "August":8,
    "September":9,
    "Oktober":10,
    "November":11,
    "Dezember":12
}

In [46]:
url = 'https://www.senatspressestelle.bremen.de/detail.php?gsid=bremen02.c.732.de&id=1464&skip=0&max=100'
baseUrl = "https://www.senatspressestelle.bremen.de/"
resp = requests.get(url)

http_encoding = resp.encoding if 'charset' in resp.headers.get('content-type', '').lower() else None
html_encoding = EncodingDetector.find_declared_encoding(resp.content, is_html=True)
encoding = html_encoding or http_encoding
locale.setlocale(locale.LC_ALL, 'de_DE')

soup = BeautifulSoup(resp.content, 'lxml', from_encoding=encoding)

In [47]:
dates = []
titles = []
urls = []

for i in soup.findAll("tr"):
    for j in i.findAll("td",{"headers":"t2"}):
        titles.append(j.text.strip())
        for link in j.find_all('a', href=True):
            urls.append(baseUrl + link['href'])
        break
    for j in i.findAll("td",{"headers":"t1"}):
        strList = j.text.strip().split(".")
        date = datetime(int(strList[2]),int(strList[1]),int(strList[0]))
        dates.append(date)
        break

print(len(dates))
print(len(titles))
print(len(urls))

100
100
100


In [48]:
for i in range(0,len(dates)):
    dbInterface.InsertNews("Bremen","Stadt Bremen","Stadtregierung",str(dates[i]),str(titles[i]),str(urls[i]),fromFedGovt = False)

In [52]:
dbInterface.cursor.execute("select count(*) from news")
for x in dbInterface.cursor:
    print(x)

(1911,)
